<a href="https://colab.research.google.com/github/julianenassimento/BRADESCO-SAUDE-em-dia/blob/main/Assistente_de_Voz_Multi_Idiomas_com_Whisper_e_Chat_GPT_BRADESCO_SA%C3%9ADE_em_dia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
language = 'pt'


#1. Instalando as dependências


In [75]:
!pip install -q openai-whisper
!pip install -q gTTS



#2. Importações

In [76]:
import whisper
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output
import base64
import io
from datetime import datetime



#3. Função para Gravar Áudio no Colab (Navegador)

In [77]:
def gravar_audio(nome_base="entrada", duracao=15):

    display(Javascript(f"""
    async function gravarAudio() {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const mediaRecorder = new MediaRecorder(stream);
      const chunks = [];
      mediaRecorder.ondataavailable = e => chunks.push(e.data);
      mediaRecorder.start();
      await new Promise(resolve => setTimeout(resolve, {duracao * 1000}));
      mediaRecorder.stop();
      await new Promise(resolve => mediaRecorder.onstop = resolve);
      const blob = new Blob(chunks, {{ type: 'audio/webm' }});
      const arrayBuffer = await blob.arrayBuffer();
      const base64Audio = btoa(
        new Uint8Array(arrayBuffer)
          .reduce((data, byte) => data + String.fromCharCode(byte), '')
      );
      google.colab.kernel.invokeFunction('notebook.salvar_audio', [base64Audio], {{}});
    }}
    gravarAudio();
    """))

    def salvar_audio(base64_audio):
        audio_bytes = base64.b64decode(base64_audio)
        with open(nome_base + ".webm", "wb") as f:
            f.write(audio_bytes)

    output.register_callback('notebook.salvar_audio', salvar_audio)



#4. Função de Transcrição (Whisper)

> Adicionar aspas



In [78]:
def transcrever_audio(arquivo="entrada.webm"):
    resultado = modelo_whisper.transcribe(arquivo)
    print("📝 Transcrição:", resultado["text"])
    return resultado["text"]



#5. Função para Falar Resposta (Áudio)

---



In [79]:
def falar(texto, nome_arquivo="resposta.mp3"):
    tts = gTTS(text=texto, lang='pt')
    tts.save(nome_arquivo)
    display(Audio(nome_arquivo, autoplay=True))



#6. Extração Simples dos Dados da Fala

In [80]:
def extrair_dados(texto):
    numeros = re.findall(r'\d+', texto)

    dados = {
        "humor": int(numeros[0]) if len(numeros) > 0 else 3,
        "sono": float(numeros[1]) if len(numeros) > 1 else 7,
        "estresse": int(numeros[2]) if len(numeros) > 2 else 3,
        "peso": float(numeros[3]) if len(numeros) > 3 else 70,
        "atividade": "sim" if "sim" in texto.lower() else "nao"
    }

    return dados


#7. Score de Saúde


In [81]:
def calcular_score(dados):
    score = 0

    if dados["humor"] >= 4:
        score += 2
    if dados["sono"] >= 7:
        score += 2
    if dados["estresse"] <= 2:
        score += 2
    if dados["atividade"] == "sim":
        score += 2

    return score


#8. Classificação


In [82]:
def classificar(score):
    if score >= 7:
        return "Baixo risco", "Parabéns! Continue mantendo sua rotina saudável."
    elif score >= 4:
        return "Risco moderado", "Atenção à sua rotina. Pequenos ajustes podem melhorar sua saúde."
    else:
        return "Alto risco", "Recomendamos acompanhamento médico e maior atenção à sua saúde."



#9. Fluxo Completo

In [83]:
def main():

    print("🏥 BRADESCO SAÚDE em Dia")
    print("Escolha como deseja responder:")
    print("1 - Digitando")
    print("2 - Por voz")

    modo = input("Digite 1 ou 2: ")

    if modo == "1":

        dados = {
            "humor": int(input("De 1 a 5, como está seu humor? ")),
            "sono": float(input("Quantas horas dormiu? ")),
            "estresse": int(input("De 1 a 5, qual seu estresse? ")),
            "atividade": input("Fez atividade física? (sim/nao) ").lower(),
            "peso": float(input("Qual seu peso? "))
        }

    elif modo == "2":

        print("🎙 Fale: humor, horas de sono, estresse, atividade física e peso.")
        gravar_audio()

        print("🎧 Áudio gravado:")
        display(Audio("entrada.webm"))

        texto = transcrever_audio()
        dados = extrair_dados(texto)

    else:
        print("Opção inválida.")
        return

    score = calcular_score(dados)
    risco, mensagem = classificar(score)

    print("\n📊 Score:", score)
    print("Classificação:", risco)
    print("Mensagem:", mensagem)

    falar(mensagem)


#10. Executar o Sistema

In [88]:
main()


🏥 BRADESCO SAÚDE em Dia
Escolha como deseja responder:
1 - Digitando
2 - Por voz
Digite 1 ou 2: 2
🎙 Fale: humor, horas de sono, estresse, atividade física e peso.


<IPython.core.display.Javascript object>

🎧 Áudio gravado:


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Transcrição:  O morro 5 na viara de sono, estresse 2, não atividade físico, 3, 55.

📊 Score: 2
Classificação: Alto risco
Mensagem: Recomendamos acompanhamento médico e maior atenção à sua saúde.
